# Overview
 In this notebook, I write the process of making a 3D model, utilizing train_labels.
 
 So, the process is summarized as follows.
 
* [0. Preparing](#0)
* [1. Extracting feature points from images](#1)
* [2. Clustering](#2)
* [3. Detecting outliers through RANSAC](#3)
* [4. Mapping fps into the 3D model](#4)
* [5. Calculating projection matrices](#5)
* [6. Getting tagets](#6)

<a id="0"></a>
# 0. Preparing

## Import

In [1]:
!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/* /root/.cache/torch/hub/checkpoints/

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl
kornia is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
  Attempting uninstall: kornia-rs
    Found existing installation: kornia_rs 0.1.3
    Uninstalling kornia_rs-0.1.3:
      Successfully uninstalled kornia_rs-0.1.3


In [2]:
import os
import numpy as np
import pandas as pd

import cv2
import torch
from lightglue import ALIKED
from lightglue.utils import load_image, rbd

from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

import warnings

## Set

In [3]:
src = "/kaggle/input/image-matching-challenge-2024"
dummy = np.zeros((1024,1024))
cv2.imwrite("dummy.png", dummy)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEBUG = len([f for f in os.listdir(f"{src}/test") if os.path.isdir(os.path.join(f"{src}/test", f))]) == 1

In [4]:
def str_to_arr(string: str, shape: tuple[int,int]):
    lst = string.split(';')
    array = []
    for s in lst:
        array.append(float(s))
    return np.array(array).reshape(shape)
def arr_to_str(array: np.array):
    return ";".join([value for value in array.reshape(-1)])

In [5]:
if DEBUG:
    num_images = 50
    max_num_keypoints = 256
    detection_threshold = 0.75
    num_feature_points = 5120
    n_clusters = 512
    random_state = 0
    main_n_init = 5
    sub_n_init = 3
    num_semi_rep_points_train = 5
    num_rep_points_train = 3
    num_train = 2000
    num_semi_rep_points_test = 5
    num_rep_points_test = 3
else:
    max_num_keypoints = 512
    detection_threshold = 0.7
    n_clusters = 1024
    random_state = 22
    main_n_init = 10
    sub_n_init = 5
    num_semi_rep_points_train = 10
    num_rep_points_test = 5
    num_train = 5000
    num_semi_rep_points_test = 10
    num_rep_points_test = 5

## Load train_labels and preprocess

In [6]:
train_labels = pd.read_csv(f"{src}/train/train_labels.csv")
train_labels.head()

,image_name,rotation_matrix,translation_vector,calibration_matrix,dataset,scene
0,00.png,0.999017467386748;-0.01951432487219089;0.03979...,-0.011700149127917355;0.018812528601332625;0.3...,5809.066058292364;0.0;2496.9582994472266;0.0;5...,transp_obj_glass_cup,transp_obj_glass_cup
1,01.png,0.999147719991382;-0.021624129414769648;0.0351...,-0.011610785964818585;0.016710808069866724;0.3...,5809.066058292364;0.0;2496.9582994472266;0.0;5...,transp_obj_glass_cup,transp_obj_glass_cup
2,02.png,0.9992527616183833;-0.02402019259931326;0.0302...,-0.011589797430545654;0.014113680489915202;0.3...,5809.066058292364;0.0;2496.9582994472266;0.0;5...,transp_obj_glass_cup,transp_obj_glass_cup
3,03.png,0.9993946226667176;-0.02356062921667625;0.0255...,-0.011471598819000773;0.011325953000912126;0.3...,5809.066058292364;0.0;2496.9582994472266;0.0;5...,transp_obj_glass_cup,transp_obj_glass_cup
4,04.png,0.9995276708105233;-0.02256816267742356;0.0208...,-0.011389007765655301;0.008237801582322509;0.3...,5809.066058292364;0.0;2496.9582994472266;0.0;5...,transp_obj_glass_cup,transp_obj_glass_cup


In [7]:
train_labels[train_labels.duplicated(subset=['image_name','dataset'],keep=False)==True]

,image_name,rotation_matrix,translation_vector,calibration_matrix,dataset,scene
78,temple_of_ba_al_shamin_palmyra_syria.png,0.7114935771373769;-0.05347611479654113;-0.700...,14.93996813428129;0.044689643146197264;-5.6992...,2313.9033196584182;0.0;1018.0;0.0;2313.9033196...,multi-temporal-temple-baalshamin,multi-temporal-temple-baalshamin
80,temple_of_ba_al_shamin_palmyra_syria.png,0.7114934016547894;-0.05348375371810413;-0.700...,14.939968046303463;0.04504033937958649;-5.6991...,2313.8720456077112;0.0;1018.0;0.0;2313.8720456...,multi-temporal-temple-baalshamin,multi-temporal-temple-baalshamin
138,temle_ba_al_shamin_4.png,0.7112621344369635;-0.05356735936347312;-0.700...,14.943852334354215;0.11347719974290106;-5.7288...,1161.2690286312848;0.0;512.0;0.0;1161.26902863...,multi-temporal-temple-baalshamin,multi-temporal-temple-baalshamin
139,temle_ba_al_shamin_4.png,0.7112612857470514;-0.05357269520714103;-0.700...,14.943858001782603;0.1129967939506197;-5.72971...,1161.2852457428498;0.0;512.0;0.0;1161.28524574...,multi-temporal-temple-baalshamin,multi-temporal-temple-baalshamin


In [8]:
train_labels.drop_duplicates(subset=['image_name','dataset'],inplace=True)
train_labels[train_labels.duplicated(subset=['image_name','dataset'],keep=False)==True]

,image_name,rotation_matrix,translation_vector,calibration_matrix,dataset,scene


In [9]:
def select_train_labels_and_paths_list_train(dataset: pd.DataFrame, num: int):
    train_labels_dataset = train_labels[train_labels.dataset==dataset].reset_index(drop=True)
    if num >= 0:
        train_labels_dataset = train_labels_dataset[:num]
    paths_list_train = src + f"/train/{dataset}/images/" + train_labels_dataset["image_name"]
    return train_labels_dataset, paths_list_train.reset_index(drop=True)

In [10]:
if DEBUG:
    dataset = "church"
    train_labels_dataset, paths_list_train = select_train_labels_and_paths_list_train(dataset, num_images)
    print(paths_list_train[:5])

0    /kaggle/input/image-matching-challenge-2024/tr...
1    /kaggle/input/image-matching-challenge-2024/tr...
2    /kaggle/input/image-matching-challenge-2024/tr...
3    /kaggle/input/image-matching-challenge-2024/tr...
4    /kaggle/input/image-matching-challenge-2024/tr...
Name: image_name, dtype: object


<a id="1"></a>
# 1. Extracting feature points from images

In [11]:
aliked = ALIKED(max_num_keypoints=max_num_keypoints,
                detection_threshold=detection_threshold).eval().to(device)
sample_image_tensor = load_image(f"{src}/train/church/images/00001.png").to(device)
sample_descriptor = aliked.extract(sample_image_tensor)["descriptors"][0]
desc_cols = []
for i in range(sample_descriptor.shape[1]):
    exec(f"desc_cols.append('d{i}')")
print(len(desc_cols))

128


In [12]:
def extract_feature_points(image_paths_list: list[str]):
    feature_points = pd.DataFrame(columns=(["image_name",'x','y']+desc_cols))
    for image_path in image_paths_list:
        image_tensor = load_image(image_path).to(device)
        features = aliked.extract(image_tensor)
        kps, descs = features["keypoints"][0], features["descriptors"][0]
        key = image_path.split('/')[-1]
        for i in range(len(kps)):
            descriptor = descs[i].numpy()
            x, y = kps[i].numpy()
            feature_points.loc[len(feature_points)] = [key, x, y] + list(descriptor)
    return feature_points

In [13]:
if DEBUG:
    feature_points_train = extract_feature_points(paths_list_train)[:num_feature_points]
    print(feature_points_train.head())
    print(len(feature_points_train))

  image_name           x           y        d0        d1        d2        d3  \
0  00059.png  531.716431  922.298340  0.101092  0.028985 -0.090774 -0.024348   
1  00059.png  563.904480  671.416321  0.000683  0.139939  0.048172 -0.021533   
2  00059.png  301.756012  365.777069  0.059662 -0.052278 -0.086119 -0.040956   
3  00059.png  308.642822  517.398926  0.059744 -0.080715 -0.110325 -0.052176   
4  00059.png  306.659241  477.777802  0.023712 -0.051453 -0.055371 -0.003278   

         d4        d5        d6  ...      d118      d119      d120      d121  \
0 -0.019952 -0.083935 -0.142644  ...  0.061926 -0.006792 -0.108350  0.128955   
1  0.028746  0.037736  0.014434  ...  0.043479 -0.099699 -0.231389  0.035314   
2  0.106852 -0.035535  0.051358  ... -0.044565 -0.153991 -0.032326  0.014585   
3  0.060690 -0.031954  0.214789  ...  0.009951 -0.103726  0.019104  0.152288   
4  0.076149 -0.087739  0.110974  ... -0.021927 -0.135376 -0.016831  0.130096   

       d122      d123      d124      d

<a id="2"></a>
# 2. Clustering

In [14]:
if DEBUG:
    X = feature_points_train.iloc[:,3:]
    print(X.head())

         d0        d1        d2        d3        d4        d5        d6  \
0  0.101092  0.028985 -0.090774 -0.024348 -0.019952 -0.083935 -0.142644   
1  0.000683  0.139939  0.048172 -0.021533  0.028746  0.037736  0.014434   
2  0.059662 -0.052278 -0.086119 -0.040956  0.106852 -0.035535  0.051358   
3  0.059744 -0.080715 -0.110325 -0.052176  0.060690 -0.031954  0.214789   
4  0.023712 -0.051453 -0.055371 -0.003278  0.076149 -0.087739  0.110974   

         d7        d8        d9  ...      d118      d119      d120      d121  \
0 -0.098850 -0.056036 -0.007138  ...  0.061926 -0.006792 -0.108350  0.128955   
1  0.119464  0.010227 -0.096795  ...  0.043479 -0.099699 -0.231389  0.035314   
2 -0.094563 -0.107459  0.018058  ... -0.044565 -0.153991 -0.032326  0.014585   
3 -0.160154 -0.072405  0.021935  ...  0.009951 -0.103726  0.019104  0.152288   
4 -0.084266 -0.097427  0.022776  ... -0.021927 -0.135376 -0.016831  0.130096   

       d122      d123      d124      d125      d126      d127  
0  0

In [15]:
main_clusterizer = KMeans(n_clusters=n_clusters,
                          random_state=random_state,
                          n_init=main_n_init)

In [16]:
def clustering_train(feature_points_train: pd.DataFrame):
    X = feature_points_train.iloc[:,3:]
    main_clusterizer.fit(X)
    labels = main_clusterizer.labels_
    feature_points_train["cluster"] = labels
    return feature_points_train

In [17]:
if DEBUG:
    feature_points_train = clustering_train(feature_points_train)
    print(feature_points_train.head())
    print(len(feature_points_train.cluster.unique()))

  image_name           x           y        d0        d1        d2        d3  \
0  00059.png  531.716431  922.298340  0.101092  0.028985 -0.090774 -0.024348   
1  00059.png  563.904480  671.416321  0.000683  0.139939  0.048172 -0.021533   
2  00059.png  301.756012  365.777069  0.059662 -0.052278 -0.086119 -0.040956   
3  00059.png  308.642822  517.398926  0.059744 -0.080715 -0.110325 -0.052176   
4  00059.png  306.659241  477.777802  0.023712 -0.051453 -0.055371 -0.003278   

         d4        d5        d6  ...      d119      d120      d121      d122  \
0 -0.019952 -0.083935 -0.142644  ... -0.006792 -0.108350  0.128955  0.016635   
1  0.028746  0.037736  0.014434  ... -0.099699 -0.231389  0.035314 -0.156107   
2  0.106852 -0.035535  0.051358  ... -0.153991 -0.032326  0.014585 -0.053627   
3  0.060690 -0.031954  0.214789  ... -0.103726  0.019104  0.152288 -0.031598   
4  0.076149 -0.087739  0.110974  ... -0.135376 -0.016831  0.130096 -0.021492   

       d123      d124      d125      d

/tmp/ipykernel_18/2099224098.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_points_train["cluster"] = labels


<a id="3"></a>
# 3. RANSAC

In [18]:
def RANSAC(feature_points: pd.DataFrame,
           num_semi_rep_points: int,
           random_state: int,
           n_init: int,
           num_rep_points: int):
    rep_feature_points_indices = []
    exception = 0
    for cluster in feature_points.cluster.unique():
        all_feature_points_cluster = feature_points[feature_points.cluster==cluster]
        X_cluster = all_feature_points_cluster.iloc[:,3:-1]
        sub_clusterizer = KMeans(n_clusters=num_semi_rep_points,
                                 random_state=random_state,
                                 n_init=n_init)
        try:
            sub_clusterizer.fit(X_cluster)
        except Exception:
            exception += 1
            continue
        centroids = sub_clusterizer.cluster_centers_
        idx_of_idx = all_feature_points_cluster.index
        dist_sums = []
        centroid_indices = []
        for centroid in centroids:
            dist_sum = 0
            dists = []
            for _, x in X_cluster.iterrows():
                dist = np.linalg.norm(x-centroid)
                dist_sum += dist
                dists.append(dist)
            dist_sums.append(dist_sum)
            centroid_index = idx_of_idx[np.argsort(dists)[0]]
            centroid_indices.append(centroid_index)
        for index in np.argsort(dist_sums)[:num_rep_points]:
            rep_feature_points_indices.append(centroid_indices[index])
    print("num of Exception :", exception)
    return rep_feature_points_indices

In [19]:
if DEBUG:
    rep_feature_points_indices = RANSAC(feature_points_train,
                                        num_semi_rep_points_train,
                                        random_state,
                                        sub_n_init,
                                        num_rep_points_train)
    rep_feature_points_train = feature_points_train.iloc[rep_feature_points_indices]
    print(rep_feature_points_train.head())
    print(len(rep_feature_points_train))

num of Exception : 27
     image_name           x           y        d0        d1        d2  \
4081  00044.png  530.362549  578.031921 -0.008815  0.046026 -0.040401   
4848  00041.png  398.476685  395.010803  0.034483  0.070868 -0.123909   
5036  00040.png  703.884399  409.047913  0.022461  0.060939 -0.005260   
1     00059.png  563.904480  671.416321  0.000683  0.139939  0.048172   
21    00059.png  563.452393  642.061462  0.050543 -0.061615 -0.001332   

            d3        d4        d5        d6  ...      d119      d120  \
4081  0.000925 -0.047770  0.030247 -0.088770  ...  0.116644 -0.167062   
4848 -0.111713  0.035571  0.026925  0.013766  ...  0.016499  0.017253   
5036 -0.040719 -0.112737  0.105812 -0.055025  ... -0.006636 -0.095534   
1    -0.021533  0.028746  0.037736  0.014434  ... -0.099699 -0.231389   
21    0.003206 -0.126384  0.022914  0.120755  ... -0.079372 -0.196310   

          d121      d122      d123      d124      d125      d126      d127  \
4081  0.063343  0.1592

<a id="4"></a>
# 4. Mapping fps into the 3D model

In [20]:
def load_projection_matrices(rep_feature_points_train: pd.DataFrame, cluster: int):
    rep_feature_points_train_cluster = rep_feature_points_train[rep_feature_points_train.cluster==cluster]
    projection_matrices_train = []
    for _, row in rep_feature_points_train_cluster.iterrows():
        image_name = row.iloc[0]
        matrices_row = train_labels_dataset[train_labels_dataset.image_name==image_name]
        rotation_matrix = str_to_arr(matrices_row.rotation_matrix.values[0], (3,3))
        translation_vector = str_to_arr(matrices_row.translation_vector.values[0], (3,1))
        calibration_matrix = str_to_arr(matrices_row.calibration_matrix.values[0], (3,3))
        if np.linalg.det(calibration_matrix) == 0:
            calibration_matrix = np.eye(3)
        projection_matrix_train = np.dot(calibration_matrix, np.hstack((rotation_matrix, translation_vector)))
        projection_matrices_train.append(projection_matrix_train)
    return projection_matrices_train

In [21]:
def rep_descriptor(rep_feature_points: pd.DataFrame, cluster: int):
    rep_feature_points_cluster = rep_feature_points[rep_feature_points.cluster==cluster]
    return rep_feature_points_cluster.iloc[:,3:-1].mean()

In [22]:
weights = []

In [23]:
def create_train_set(projection_matrices_train: list, num_train: int):
    train_set_cols = []
    for i in range(len(projection_matrices_train)):
        exec(f"train_set_cols += ['x{i}','y{i}']")
    train_2D_vector_sets = pd.DataFrame(columns=train_set_cols)
    train_3D_vectors = np.hstack((100*np.random.rand(num_train,3),np.ones((num_train,1))))
    for y_vector in train_3D_vectors:
        X_vector = []
        for projection_matrix_train in projection_matrices_train:
            x, y, w = np.dot(projection_matrix_train, y_vector.T).T
            X_vector += [x/w, y/w]
            weights.append(w)
        train_2D_vector_sets.loc[len(train_2D_vector_sets)] = X_vector
    return train_2D_vector_sets, train_3D_vectors, weights

In [24]:
def get_3D_coordinates(test_X, projection_matrices_train: list[np.array]):
    train_X, train_y, weights = create_train_set(projection_matrices_train, num_train)
    linear_model = LinearRegression(fit_intercept=False)
    linear_model.fit(train_X, train_y)
    x, y, z, _ = linear_model.predict(test_X)[0]
    return cluster, x, y, z

In [25]:
if DEBUG:
    model = pd.DataFrame(columns=(["cluster",'x','y','z']))
    warnings.filterwarnings("ignore")
    for cluster in rep_feature_points_train.cluster.unique():
        test_X = rep_feature_points_train[rep_feature_points_train.cluster==cluster].iloc[:,1:3].values.reshape((1,-1))
        projection_matrices_train = load_projection_matrices(rep_feature_points_train, cluster)
        model.loc[len(model)] = get_3D_coordinates(test_X, projection_matrices_train)
    print(model.head())

   cluster          x          y         z
0     14.0  43.301679   7.476143  3.281653
1    340.0   3.039612   1.127877  0.205649
2    435.0   0.165227   0.121317  0.081439
3    454.0   3.936379   1.894957  0.980333
4    166.0  39.730419  17.363583  9.693573


# (hidden)Preparing Submission

In [26]:
sample_submission = pd.read_csv(f"{src}/sample_submission.csv")
sample_submission.head()

,image_path,dataset,scene,rotation_matrix,translation_vector
0,test/church/images/00046.png,church,church,0.9711190310868889;0.8911359604510818;0.532623...,0.11313904678659892;0.8432251602872274;0.42609...
1,test/church/images/00090.png,church,church,0.20360069697426775;0.6504946118853364;0.86491...,0.680480221630539;0.8028143635199276;0.5028546...
2,test/church/images/00092.png,church,church,0.831738872848693;0.5821652805589269;0.6032012...,0.16253346680799086;0.2541505473736173;0.87383...
3,test/church/images/00087.png,church,church,0.1604600521094205;0.6489370807852337;0.721831...,0.2161246142041484;0.9530634517785977;0.449099...
4,test/church/images/00050.png,church,church,0.38660496248295817;0.4015767283826711;0.92385...,0.16786019693458032;0.8533039000844846;0.96215...


In [27]:
def parse_sample_submission():
    data_dict = {}
    sample_submission = pd.read_csv(f"{src}/sample_submission.csv")
    header = sample_submission.columns
    print(f"header: {header}")
    for i, l in sample_submission.iterrows():
        image_path, dataset, scene, _, _ = l
        if dataset not in data_dict:
            data_dict[dataset] = []
        data_dict[dataset].append(f"{src}/{image_path}")
    for dataset in data_dict:
        print(f"{dataset} -> {len(data_dict[dataset])} images")
    return data_dict, header

<a id="5"></a>
# 5. Calculating projection matrices

In [28]:
def clustering_test(feature_points_test: pd.DataFrame):
    X = feature_points_test.iloc[:,3:]
    labels = main_clusterizer.predict(X)
    feature_points_test["cluster"] = labels
    return feature_points_test

In [29]:
if DEBUG:
    data_dict, _ = parse_sample_submission()
    dataset = "church"
    paths_list_test = data_dict[dataset][:num_images]
    feature_points_test = extract_feature_points(paths_list_test)[:num_feature_points]
    feature_points_test = clustering_test(feature_points_test)
    rep_feature_points_indices_test = RANSAC(feature_points_test,
                                             num_semi_rep_points_test,
                                             random_state,
                                             sub_n_init,
                                             num_rep_points_test)
    rep_feature_points_test = feature_points_test.iloc[rep_feature_points_indices_test]
    print(rep_feature_points_test.head())
    print(len(rep_feature_points_test))

header: Index(['image_path', 'dataset', 'scene', 'rotation_matrix',
       'translation_vector'],
      dtype='object')
church -> 41 images
num of Exception : 134
     image_name           x           y        d0        d1        d2  \
0     00046.png  304.146667  554.505737 -0.049578 -0.226732 -0.081279   
4560  00008.png  603.916870  358.267273  0.014216  0.015977 -0.131726   
3453  00024.png  669.453613  394.991791  0.078373 -0.054839  0.004154   
2463  00081.png  239.017563  247.155716 -0.070351 -0.104196 -0.101436   
1     00046.png  516.795959  105.586243 -0.051849 -0.079518 -0.308859   

            d3        d4        d5        d6  ...      d119      d120  \
0     0.135531 -0.098278 -0.024081  0.010061  ...  0.000400 -0.025382   
4560  0.131950 -0.012961 -0.006080 -0.010256  ...  0.007103 -0.010005   
3453  0.096712 -0.060796 -0.142891 -0.065903  ...  0.135146 -0.116231   
2463 -0.062865  0.146592 -0.054390  0.036865  ... -0.205398  0.265364   
1     0.105809  0.171159 -0.01616

In [30]:
def median(lst):
    sorted_lst = sorted(lst)
    n = len(sorted_lst)
    if n % 2 == 1:
        return sorted_lst[n // 2]
    else:
        mid = n // 2
        return (sorted_lst[mid - 1] + sorted_lst[mid]) / 2

In [31]:
weight = median(weights)

In [32]:
def get_projection_matrix(model: pd.DataFrame, rep_feature_points_test: pd.DataFrame, image_name: str):
    rep_fps_test_img = rep_feature_points_test[rep_feature_points_test.image_name==image_name]
    X, y = [], []
    for cluster in rep_fps_test_img.cluster:
        rep_fps_test_img_cluster = rep_fps_test_img[rep_fps_test_img.cluster==cluster]
        try:
            coordinates_3D = model[model.cluster==cluster].iloc[0,1:]
        except:
            pass
        coordinates_2D = rep_fps_test_img_cluster.iloc[np.random.choice(range(len(rep_fps_test_img_cluster))),1:3]
        x0, y0, z0 = coordinates_3D
        x1, y1 = coordinates_2D
        X.append([x0, y0, z0, 1])
        y.append([x1, y1, weight])
    linear_model = LinearRegression(fit_intercept=False)
    try:
        linear_model.fit(X, y)
        return linear_model.coef_
    except:
        return np.hstack((np.eye(3), np.ones((3,1))))

In [33]:
if DEBUG:
    projection_matrices_test = pd.DataFrame(columns=["image_name","projection_matrix"])
    for image_path in paths_list_test:
        image_name = image_path.split('/')[-1]
        projection_matrix_test = get_projection_matrix(model,
                                                       rep_feature_points_test,
                                                       image_name)
        projection_matrices_test.loc[len(projection_matrices_test)] = [image_name, projection_matrix_test]
    print(projection_matrices_test.head())
    print(len(projection_matrices_test))
    print(projection_matrices_test.projection_matrix[0])

  image_name                                  projection_matrix
0  00046.png  [[1.8311012698590675e-09, -1.7655273573948983e...
1  00090.png  [[2.965101450573964e-08, -2.0578422194779173e-...
2  00092.png  [[0.24328415332160147, -0.09283607983516792, -...
3  00087.png  [[-0.04854749632460627, 0.00901805044710112, 0...
4  00050.png  [[8.707769090696845e-11, -4.677030460794069e-1...
41
[[ 1.83110127e-09 -1.76552736e-09  2.82125685e-10  4.23807236e+02]
 [ 1.55161635e-10 -5.43750353e-10  4.50790140e-10  6.10401104e+02]
 [ 6.57626634e-25 -8.91804098e-25  2.64148496e-25  6.57849268e+01]]
